In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

!pip install statsmodels
import statsmodels.api as sm
import statsmodels.formula.api as sf

import pylogit as pl
from collections import OrderedDict

In [45]:
df=pd.read_csv('conjoint_tablet_data.csv')
df['Price']=df['Price'].str[1:]
df['Price']=df['Price'].astype('int')
df.head(5)

,ConsumerId,ChoiceSetId,AlternativeIdInSet,Choice,Brand,Size,Storage,Ram,Battery,Price
0,1,1,1,1,iPad,sz7inch,st32gb,r4gb,b7h,499
1,1,1,2,0,Surface,sz10inch,st64gb,r2gb,b9h,399
2,1,1,3,0,Kindle,sz9inch,st16gb,r2gb,b8h,499
3,1,2,1,1,iPad,sz8inch,st32gb,r1gb,b8h,399
4,1,2,2,0,Surface,sz10inch,st128gb,r4gb,b7h,299


In [51]:
df=pd.get_dummies(df,columns=['Brand','Size','Storage','Ram','Battery']).astype('int')
df.head(5)

,ConsumerId,ChoiceSetId,AlternativeIdInSet,Choice,Price,Brand_Galaxy,Brand_Kindle,Brand_Nexus,Brand_Surface,Brand_iPad,...,Storage_st128gb,Storage_st16gb,Storage_st32gb,Storage_st64gb,Ram_r1gb,Ram_r2gb,Ram_r4gb,Battery_b7h,Battery_b8h,Battery_b9h
0,1,1,1,1,499,0,0,0,0,1,...,0,0,1,0,0,0,1,1,0,0
1,1,1,2,0,399,0,0,0,1,0,...,0,0,0,1,0,1,0,0,0,1
2,1,1,3,0,499,0,1,0,0,0,...,0,1,0,0,0,1,0,0,1,0
3,1,2,1,1,399,0,0,0,0,1,...,0,0,1,0,1,0,0,0,1,0
4,1,2,2,0,299,0,0,0,1,0,...,1,0,0,0,0,0,1,1,0,0


In [52]:
df.drop(['Brand_Nexus','Size_sz7inch','Storage_st16gb','Battery_b7h','Ram_r1gb'],axis=1,inplace=True)

In [53]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6165 entries, 0 to 6164
Data columns (total 19 columns):
ConsumerId            6165 non-null int32
ChoiceSetId           6165 non-null int32
AlternativeIdInSet    6165 non-null int32
Choice                6165 non-null int32
Price                 6165 non-null int32
Brand_Galaxy          6165 non-null int32
Brand_Kindle          6165 non-null int32
Brand_Surface         6165 non-null int32
Brand_iPad            6165 non-null int32
Size_sz10inch         6165 non-null int32
Size_sz8inch          6165 non-null int32
Size_sz9inch          6165 non-null int32
Storage_st128gb       6165 non-null int32
Storage_st32gb        6165 non-null int32
Storage_st64gb        6165 non-null int32
Ram_r2gb              6165 non-null int32
Ram_r4gb              6165 non-null int32
Battery_b8h           6165 non-null int32
Battery_b9h           6165 non-null int32
dtypes: int32(19)
memory usage: 457.7 KB


In [54]:
variable=df.iloc[:,4:].columns
variable

Index(['Price', 'Brand_Galaxy', 'Brand_Kindle', 'Brand_Surface', 'Brand_iPad',
       'Size_sz10inch', 'Size_sz8inch', 'Size_sz9inch', 'Storage_st128gb',
       'Storage_st32gb', 'Storage_st64gb', 'Ram_r2gb', 'Ram_r4gb',
       'Battery_b8h', 'Battery_b9h'],
      dtype='object')

In [55]:
basic_specification=OrderedDict()
basic_names=OrderedDict()

In [56]:
for col in variable:
    basic_specification[col]=[[1,2,3]] 
    basic_names[col]=[col]

In [57]:
basic_specification

OrderedDict([('Price', [[1, 2, 3]]),
             ('Brand_Galaxy', [[1, 2, 3]]),
             ('Brand_Kindle', [[1, 2, 3]]),
             ('Brand_Surface', [[1, 2, 3]]),
             ('Brand_iPad', [[1, 2, 3]]),
             ('Size_sz10inch', [[1, 2, 3]]),
             ('Size_sz8inch', [[1, 2, 3]]),
             ('Size_sz9inch', [[1, 2, 3]]),
             ('Storage_st128gb', [[1, 2, 3]]),
             ('Storage_st32gb', [[1, 2, 3]]),
             ('Storage_st64gb', [[1, 2, 3]]),
             ('Ram_r2gb', [[1, 2, 3]]),
             ('Ram_r4gb', [[1, 2, 3]]),
             ('Battery_b8h', [[1, 2, 3]]),
             ('Battery_b9h', [[1, 2, 3]])])

In [60]:
model1=pl.create_choice_model(data=df,
                                    alt_id_col='AlternativeIdInSet',
                                    obs_id_col='ChoiceSetId',
                                    choice_col='Choice',
                                    specification=basic_specification,
                                    model_type="MNL",
                                    names=basic_names)

D:\Program Files\Data Science\Anaconda\lib\site-packages\pylogit\choice_tools.py:703: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  design_matrix = np.hstack((x[:, None] for x in independent_vars))


In [61]:
model1.fit_mle(np.zeros(15))
model1.get_statsmodels_summary()

Log-likelihood at zero: -2,257.6483
Initial Log-likelihood: -2,257.6483


D:\Program Files\Data Science\Anaconda\lib\site-packages\scipy\optimize\_minimize.py:505: RuntimeWarning: Method BFGS does not use Hessian information (hess).
  RuntimeWarning)


Estimation Time for Point Estimation: 0.08 seconds.
Final log-likelihood: -1,938.8948


<class 'statsmodels.iolib.summary.Summary'>
"""
                     Multinomial Logit Model Regression Results                    
===================================================================================
Dep. Variable:                      Choice   No. Observations:                2,055
Model:             Multinomial Logit Model   Df Residuals:                    2,040
Method:                                MLE   Df Model:                           15
Date:                     Mon, 27 Apr 2020   Pseudo R-squ.:                   0.141
Time:                             08:38:49   Pseudo R-bar-squ.:               0.135
AIC:                             3,907.790   Log-Likelihood:             -1,938.895
BIC:                             3,992.210   LL-Null:                    -2,257.648
===================================================================================
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Price              -0.0051      0.000    -18.489      0.000      -0.006      -0.005
Brand_Galaxy        0.3379      0.093      3.653      0.000       0.157       0.519
Brand_Kindle        0.2630      0.100      2.640      0.008       0.068       0.458
Brand_Surface       0.1450      0.094      1.545      0.122      -0.039       0.329
Brand_iPad          0.9780      0.094     10.434      0.000       0.794       1.162
Size_sz10inch       0.3241      0.084      3.849      0.000       0.159       0.489
Size_sz8inch        0.1891      0.083      2.280      0.023       0.027       0.352
Size_sz9inch        0.4355      0.081      5.388      0.000       0.277       0.594
Storage_st128gb     0.5898      0.087      6.775      0.000       0.419       0.760
Storage_st32gb      0.2169      0.083      2.615      0.009       0.054       0.379
Storage_st64gb      0.5782      0.081      7.154      0.000       0.420       0.737
Ram_r2gb            0.3189      0.067      4.742      0.000       0.187       0.451
Ram_r4gb            0.6357      0.065      9.853      0.000       0.509       0.762
Battery_b8h         0.1300      0.065      1.995      0.046       0.002       0.258
Battery_b9h         0.1254      0.065      1.927      0.054      -0.002       0.253
===================================================================================
"""

In [64]:
df['predicted']=model1.predict(df)
choice_number=len(df['ChoiceSetId'].unique())
df['predicted_choice']=0

for i in range(1,choice_number+1):
        t=0
        t=df['predicted'].where(df['ChoiceSetId']==i).max()
        df['predicted_choice'][df['ChoiceSetId']==i]=df['predicted'].where(df['ChoiceSetId']==i).apply(lambda x:1 if x==t else 0)
        


D:\Program Files\Data Science\Anaconda\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,ConsumerId,ChoiceSetId,AlternativeIdInSet,Choice,Price,Brand_Galaxy,Brand_Kindle,Brand_Surface,Brand_iPad,Size_sz10inch,...,Size_sz9inch,Storage_st128gb,Storage_st32gb,Storage_st64gb,Ram_r2gb,Ram_r4gb,Battery_b8h,Battery_b9h,predicted,predicted_choice
0,1,1,1,1,499,0,0,0,1,0,...,0,0,1,0,0,1,0,0,0.371726,0
1,1,1,2,0,399,0,0,1,0,1,...,0,0,0,1,1,0,0,1,0.440552,1
2,1,1,3,0,499,0,1,0,0,0,...,1,0,0,0,1,0,1,0,0.187722,0
3,1,2,1,1,399,0,0,0,1,0,...,0,0,1,0,0,0,1,0,0.236780,0
4,1,2,2,0,299,0,0,1,0,1,...,0,1,0,0,0,1,0,0,0.471862,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6160,137,2054,2,1,499,0,0,0,1,0,...,0,0,0,1,1,0,1,0,0.282025,0
6161,137,2054,3,0,199,1,0,0,0,0,...,0,0,1,0,0,0,0,1,0.345032,0
6162,137,2055,1,0,399,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0.239909,0
6163,137,2055,2,0,399,0,0,0,1,0,...,0,0,0,1,1,0,1,0,0.307516,0


In [65]:
df.head(20)

,ConsumerId,ChoiceSetId,AlternativeIdInSet,Choice,Price,Brand_Galaxy,Brand_Kindle,Brand_Surface,Brand_iPad,Size_sz10inch,...,Size_sz9inch,Storage_st128gb,Storage_st32gb,Storage_st64gb,Ram_r2gb,Ram_r4gb,Battery_b8h,Battery_b9h,predicted,predicted_choice
0,1,1,1,1,499,0,0,0,1,0,...,0,0,1,0,0,1,0,0,0.371726,0
1,1,1,2,0,399,0,0,1,0,1,...,0,0,0,1,1,0,0,1,0.440552,1
2,1,1,3,0,499,0,1,0,0,0,...,1,0,0,0,1,0,1,0,0.187722,0
3,1,2,1,1,399,0,0,0,1,0,...,0,0,1,0,0,0,1,0,0.236780,0
4,1,2,2,0,299,0,0,1,0,1,...,0,1,0,0,0,1,0,0,0.471862,1
5,1,2,3,0,199,0,0,0,0,0,...,0,0,0,1,0,0,0,1,0.291358,0
6,1,3,1,0,199,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0.476087,1
7,1,3,2,0,399,0,1,0,0,0,...,1,0,0,0,0,1,0,1,0.297432,0
8,1,3,3,1,499,1,0,0,0,1,...,0,1,0,0,1,0,0,1,0.226481,0
9,1,4,1,1,299,0,0,0,1,0,...,1,0,0,0,0,1,0,0,0.373037,0


In [3]:
lm=sf.mnlogit('Choice ~ Brand + Size + Storage + Ram + Battery + Price',data=df).fit()
print(lm.summary())


Optimization terminated successfully.
         Current function value: 0.579977
         Iterations 5
                          MNLogit Regression Results                          
Dep. Variable:                 Choice   No. Observations:                 6165
Model:                        MNLogit   Df Residuals:                     6149
Method:                           MLE   Df Model:                           15
Date:                Thu, 16 Apr 2020   Pseudo R-squ.:                 0.08882
Time:                        08:06:13   Log-Likelihood:                -3575.6
converged:                       True   LL-Null:                       -3924.1
Covariance Type:            nonrobust   LLR p-value:                7.721e-139
         Choice=1       coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept             0.5084      0.132      3.864      0.000       0.251       0.766
Brand[T.

In [3]:
!pip install choicemodels

In [11]:
t=class choicemodels.MultinomialLogit(data=df,model_expression='Choice ~ 0 + Brand + Size + Storage + Ram + Battery + Price',choice_col=choice,observation_id_col=obs).fit()

SyntaxError: invalid syntax (<ipython-input-11-15cac8ba100b>, line 1)

In [10]:
choice=df['Choice']
obs=df['AlternativeIdInSet']

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6165 entries, 0 to 6164
Data columns (total 10 columns):
ConsumerId            6165 non-null int64
ChoiceSetId           6165 non-null int64
AlternativeIdInSet    6165 non-null int64
Choice                6165 non-null int64
Brand                 6165 non-null object
Size                  6165 non-null object
Storage               6165 non-null object
Ram                   6165 non-null object
Battery               6165 non-null object
Price                 6165 non-null int32
dtypes: int32(1), int64(4), object(5)
memory usage: 457.7+ KB


In [ ]:
https://github.com/timothyb0912/pylogit/blob/master/examples/notebooks/Main%20PyLogit%20Example.ipynb

In [4]:
df=pd.get_dummies(df,columns=['Brand','Size','Storage','Ram','Battery'])

In [5]:
df

,ConsumerId,ChoiceSetId,AlternativeIdInSet,Choice,Price,Brand_Galaxy,Brand_Kindle,Brand_Nexus,Brand_Surface,Brand_iPad,...,Storage_st128gb,Storage_st16gb,Storage_st32gb,Storage_st64gb,Ram_r1gb,Ram_r2gb,Ram_r4gb,Battery_b7h,Battery_b8h,Battery_b9h
0,1,1,1,1,499,0,0,0,0,1,...,0,0,1,0,0,0,1,1,0,0
1,1,1,2,0,399,0,0,0,1,0,...,0,0,0,1,0,1,0,0,0,1
2,1,1,3,0,499,0,1,0,0,0,...,0,1,0,0,0,1,0,0,1,0
3,1,2,1,1,399,0,0,0,0,1,...,0,0,1,0,1,0,0,0,1,0
4,1,2,2,0,299,0,0,0,1,0,...,1,0,0,0,0,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6160,137,2054,2,1,499,0,0,0,0,1,...,0,0,0,1,0,1,0,0,1,0
6161,137,2054,3,0,199,1,0,0,0,0,...,0,0,1,0,1,0,0,0,0,1
6162,137,2055,1,0,399,0,0,0,0,1,...,1,0,0,0,1,0,0,1,0,0
6163,137,2055,2,0,399,0,0,0,0,1,...,0,0,0,1,0,1,0,0,1,0


In [15]:
import pylogit as pl
from collections import OrderedDict

In [16]:
choice_spec=OrderedDict()
choice_name=OrderedDict()

In [20]:
df=df.pivot(index='ConsumerId')

KeyError: 'None of [None] are in the columns'

In [8]:
Y=df['Choice']
X=df.iloc[:,4:]
test=sfg

AttributeError: module 'statsmodels.api' has no attribute 'logit'

In [14]:
names=df.iloc[:,4:]